<a href="https://colab.research.google.com/github/Twinkle0799/Predicton-of-House-prices-using-tensorflow/blob/master/Tensorflow%20house%20price%20predicting%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 TASK 1 : **IMPORTING LIBRARIES**

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
#from utils import *
import sys
sys.path.append('PATH_TO_TENSORFLOW_OBJECT_DETECTION_FOLDER')
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from keras.callbacks import ModelCheckpoint, EarlyStopping


%matplotlib inline
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

print('Libraries Imported')

Libraries Imported


TASK 2 : **IMPORTING THE DATA**

In [4]:
from google.colab import files
uploaded = files.upload()
#To upload from your local drive

Saving data.csv to data.csv


In [5]:
df = pd.read_csv('data.csv')
df.columns = [' ','data','age','distance','stores','latitude','longitude','price']
df.head()

,,data,age,distance,stores,latitude,longitude,price
0,1,2007,4,2,3,86,121,12032
1,2,2016,18,3,7,90,120,13560
2,3,2002,13,2,2,80,128,12029
3,4,2014,25,5,8,81,122,14157
4,5,2008,11,9,3,88,126,14287


CHECK MISSING DATA : Its  good practice to check if the data has any missing values. 

In [6]:
df.isna().sum

<bound method DataFrame.sum of               data    age  distance  stores  latitude  longitude  price
0     False  False  False     False   False     False      False  False
1     False  False  False     False   False     False      False  False
2     False  False  False     False   False     False      False  False
3     False  False  False     False   False     False      False  False
4     False  False  False     False   False     False      False  False
...     ...    ...    ...       ...     ...       ...        ...    ...
4994  False  False  False     False   False     False      False  False
4995  False  False  False     False   False     False      False  False
4996  False  False  False     False   False     False      False  False
4997  False  False  False     False   False     False      False  False
4998  False  False  False     False   False     False      False  False

[4999 rows x 8 columns]>

TASK 3 : **DATA NORMALIZATION**

In [7]:
df = df.iloc[:, 1:]
df_norm = (df - df.mean())/df.std()
df_norm.head()


,data,age,distance,stores,latitude,longitude,price
0,-0.350447,-1.318954,-0.930413,-0.609189,0.325211,-1.261125,-1.836255
1,1.298471,-0.083366,-0.617878,0.663413,1.590122,-1.576801,-0.339484
2,-1.266513,-0.524647,-0.930413,-0.927340,-1.572157,0.948607,-1.839193
3,0.932045,0.534428,0.007190,0.981564,-1.255929,-0.945449,0.245315
4,-0.167234,-0.701160,1.257326,-0.609189,0.957666,0.317255,0.372658


In [8]:
#convert label values
y_mean=df['price'].mean()
y_std=df['price'].std()
def convert_label_value(pred):
  return int(pred*y_std+y_mean)
print(convert_label_value(-1.836255))

12031


TASK 4 : **TRAINING AND TEST SET**

In [9]:
x = df_norm.iloc[:, :6]
x.head()

,data,age,distance,stores,latitude,longitude
0,-0.350447,-1.318954,-0.930413,-0.609189,0.325211,-1.261125
1,1.298471,-0.083366,-0.617878,0.663413,1.590122,-1.576801
2,-1.266513,-0.524647,-0.930413,-0.927340,-1.572157,0.948607
3,0.932045,0.534428,0.007190,0.981564,-1.255929,-0.945449
4,-0.167234,-0.701160,1.257326,-0.609189,0.957666,0.317255


In [10]:
y=df_norm.iloc[:,-1]
y.head()

0   -1.836255
1   -0.339484
2   -1.839193
3    0.245315
4    0.372658
Name: price, dtype: float64

In [11]:
#feature and label values
x_arr = x.values
y_arr = y.values
print('feature array shape:',x_arr.shape)
print('label array shape: ',y_arr.shape)

feature array shape: (4999, 6)
label array shape:  (4999,)


In [12]:
#train and test split
x_train,x_test,y_train,y_test = train_test_split(x_arr,y_arr,test_size=0.05,random_state=0)
print('Training Set: ',x_train.shape,y_train.shape)
print('Test Set :',x_test.shape,y_test.shape)

Training Set:  (4749, 6) (4749,)
Test Set : (250, 6) (250,)


TASK 5 : **CREATE THE MODEL**

In [13]:
def get_model():
  model=Sequential([Dense(10,input_shape=(6, ),activation = 'relu'),
                    Dense(20,activation='relu'),Dense(1,activation='relu'),Dense(1)])
  model.compile(loss='mse',optimizer ='adam')
  return model
get_model().summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                70        
_________________________________________________________________
dense_1 (Dense)              (None, 20)                220       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 2         
Total params: 313
Trainable params: 313
Non-trainable params: 0
_________________________________________________________________


TASK : 6 **MODEL TRAINING**

In [0]:
es_cb = EarlyStopping(monitor= 'val_loss', patience = 5)
model = get_model()
preds_on_untrained = model.predict(x_test)
history=model.fit(x_train,y_train,validation_data =(x_test,y_test), epochs = 100, callabacks = [es_cb])

In [0]:
plot_loss(history)

**PLOT RAW PREDICTIONS**

In [0]:
preds_on_trained = model.predict(x_test)
compare_predictions(preds_on_untrained,preds_on_trained,y_test)

In [0]:
price_untrained = [convert_label_value(y) for y in preds_on_untrained]
price_trained=[convert_label_value(y) for y in preds_on_trained]
price_test = [convert_label_value(y) for y in y_test]
compare_predictions(price_untrained,price_trained,price_test)